### 코드 종속성
- python 3.11
- opencv-python
- mediapipe
- numpy
- 개쩌는 GPU

---

#### 할일
- GPU 사용할 수 있게 열심히 노력하기

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
from mediapipe.framework.formats import landmark_pb2

# MediaPipe 관련 클래스 및 함수 임포트
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

# 전역 변수로 탐지 결과를 저장할 객체 선언
# 콜백 함수와 메인 루프 간의 데이터 전달을 위해 사용
detection_result = None

# MediaPipe가 탐지를 완료할 때마다 호출될 콜백 함수
def result_callback(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    """
    LIVE_STREAM 모드에서 탐지 결과를 비동기적으로 받는 콜백 함수.
    
    Args:
      result: HandLandmarker의 탐지 결과.
      output_image: 처리된 이미지 (여기서는 사용하지 않음).
      timestamp_ms: 탐지가 수행된 시간.
    """
    global detection_result
    detection_result = result

# HandLandmarker 옵션 설정
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path="D:\work space\lean_startup\hand_landmarker.task"), # 모델 파일 경로
    running_mode=VisionRunningMode.LIVE_STREAM, # 실시간 스트림 모드 설정
    num_hands=2, # 최대 탐지할 손의 개수
    result_callback=result_callback) # 결과 콜백 함수 지정

# 웹캠 설정
cap = cv2.VideoCapture(0)

# HandLandmarker 인스턴스 생성
with HandLandmarker.create_from_options(options) as landmarker:
    if not cap.isOpened():
        print("카메라를 열 수 없습니다.")
        exit()

    while True:
        # 웹캠에서 프레임 읽기
        ret, frame = cap.read()
        if not ret:
            print("프레임을 읽을 수 없습니다.")
            break

        # OpenCV 프레임(BGR)을 MediaPipe Image 객체(SRGB)로 변환
        # MediaPipe는 RGB 순서를 사용하므로 변환이 필요합니다.
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

        # 현재 시간(밀리초)을 타임스탬프로 사용
        frame_timestamp_ms = int(time.time() * 1000)

        # 비동기적으로 손 랜드마크 탐지 수행
        # 이 함수는 결과를 반환하지 않고, 탐지가 완료되면 콜백 함수를 호출함
        landmarker.detect_async(mp_image, frame_timestamp_ms)

        # 화면에 결과 표시
        # 콜백 함수에 의해 detection_result가 업데이트되었는지 확인
        if detection_result and detection_result.hand_landmarks:
            # 원본 프레임을 복사하여 그 위에 랜드마크를 그림
            annotated_image = frame.copy()
            
            # 모든 탐지된 손에 대해 랜드마크와 연결선을 그림
            for hand_landmarks in detection_result.hand_landmarks:
                
                # 1. draw_landmarks가 요구하는 형식의 객체를 생성합니다.
                hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
                
                # 2. 파이썬 리스트에 있는 랜드마크들을 새로 만든 객체에 추가합니다.
                hand_landmarks_proto.landmark.extend([
                    landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
                ])

                # 3. 변환된 객체(hand_landmarks_proto)를 draw_landmarks 함수에 전달합니다.
                mp.solutions.drawing_utils.draw_landmarks(
                    annotated_image,
                    hand_landmarks_proto,  # <-- 바로 이 부분입니다!
                    mp.solutions.hands.HAND_CONNECTIONS,
                    mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                    mp.solutions.drawing_styles.get_default_hand_connections_style())
                
            # 랜드마크가 그려진 이미지를 화면에 표시
            cv2.imshow('Hand Landmarks', annotated_image)
        else:
            # 탐지된 손이 없으면 원본 프레임을 그대로 표시
            cv2.imshow('Hand Landmarks', frame)

        # 'q' 키를 누르면 루프 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# 자원 해제
cap.release()
cv2.destroyAllWindows()